<a href="https://colab.research.google.com/github/Arifanalysis/Fabric-class/blob/main/novel_classes_AT_MLNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d sobouhiarif/fabric-classifications
#kaggle datasets download -d sobouhiarif/fabric-defect-test

In [ ]:
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import zipfile

# Specify the path to the zip file
zip_file_path = '/content/fabric-classifications.zip'

# Specify the directory where you want to extract the contents
extract_path = '/content/Fabric FDD/Fabric_classification'

# Unzip the folder
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

In [ ]:
from google.colab import files
import shutil
folder_path = "/content/Fabric FDD/Fabric_classification"
zip_file_path = "/content/fabric-classifications.zip"
shutil.make_archive(zip_file_path[:-4], 'zip', folder_path)

In [ ]:
import os
import cv2
import numpy as np
import time
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization, GlobalAveragePooling2D
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import matplotlib.pyplot as plt
from keras.layers import TimeDistributed
from keras.applications import MobileNet
from keras import backend as K
from keras.layers import Layer
import tensorflow as tf

# Attention layer class
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight', shape=(input_shape[-1], 1),
                                 initializer='glorot_uniform', trainable=True)
        self.b = self.add_weight(name='attention_bias', shape=(input_shape[1], 1),
                                 initializer='glorot_uniform', trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

# Function to load the dataset with hierarchical structure
def load_dataset():
    base_dir = '/content/Fabric FDD/Fabric_classification/Fabric classification'
    # Define hierarchical categories for Weaving and Knitting
    subcategories = {
        'Weaving_gray': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
        'Weaving_dyed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
        'Weaving_printed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
        'Knitting_gray': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
        'Knitting_dyed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
        'Knitting_printed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective']
    }

    images = []
    labels = []
    categories = list(subcategories.keys())
    category_counts = {category: 0 for category in categories}

    for label, category in enumerate(categories):
        for defect in subcategories[category]:
            category_dir = os.path.join(base_dir, category, defect)
            if not os.path.exists(category_dir):
                print(f"Directory {category_dir} does not exist. Skipping this category.")
                continue
            for filename in os.listdir(category_dir):
                if filename.endswith('.jpg'):
                    img = cv2.imread(os.path.join(category_dir, filename))
                    img = cv2.resize(img, (128, 128))  # Resize image to 128x128
                    images.append(img)
                    labels.append(label)
                    category_counts[category] += 1  # Count the images for each category

    images = np.array(images)
    labels = np.array(labels)

    print("Image counts per category:", category_counts)  # Print the counts for debugging

    return images, labels, categories

# Load the dataset
images, labels, categories = load_dataset()

# Ensure there are images loaded
if len(images) == 0:
    raise ValueError("No images found. Please check the dataset path and ensure images are available.")

# Preprocess the images (normalize pixel values)
images = images.astype('float32') / 255.0

# Convert labels to one-hot encoding
labels = to_categorical(labels, num_classes=len(categories))

# Reshape images for LSTM input
images = images.reshape(images.shape[0], 1, 128, 128, 3)

# Split the dataset into training and test sets
trainX, testX, trainY, testY = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define the model
model = Sequential()

# Load pre-trained MobileNet without the top layers
feature_extractor = MobileNet(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Freeze the weights of the pre-trained model
for layer in feature_extractor.layers:
    layer.trainable = False

# Use TimeDistributed to apply the MobileNet to each image in the sequence
model.add(TimeDistributed(feature_extractor, input_shape=(1, 128, 128, 3)))
model.add(TimeDistributed(GlobalAveragePooling2D()))

# Add LSTM layer
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.5))

# Add Attention layer
model.add(AttentionLayer())

# Add Dense layers for classification
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# Output layer for final classification
model.add(Dense(len(categories), activation='softmax', kernel_regularizer='l2'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
print(model.summary())

# Callbacks
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
start_time = time.time()
history = model.fit(trainX, trainY, epochs=90, batch_size=32, validation_data=(testX, testY), callbacks=[lr_reduction, early_stopping], verbose=1)
end_time = time.time()

# Evaluate the model
loss, accuracy = model.evaluate(testX, testY, verbose=0)
print('Test Loss:', loss)
print('Test Accuracy:', accuracy)

# Get predictions and evaluate metrics
predY = model.predict(testX)
predY_classes = np.argmax(predY, axis=1)
trueY_classes = np.argmax(testY, axis=1)

unique_true_labels = np.unique(trueY_classes)
class_report = classification_report(
    trueY_classes, predY_classes,
    target_names=[categories[i] for i in unique_true_labels],
    output_dict=True,
    zero_division=0
)
print(class_report)

conf_matrix = confusion_matrix(trueY_classes, predY_classes)
print(conf_matrix)

accuracy_scores = accuracy_score(trueY_classes, predY_classes)
f1_scores = f1_score(trueY_classes, predY_classes, average=None)
print("Accuracy Scores: ", accuracy_scores)
print("F1 Scores: ", f1_scores)
print("Average F1 Score: ", np.mean(f1_scores))

print(f"Total Training Time: {end_time - start_time} seconds")

# Plot training & validation loss and accuracy
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

# Visualize test images
plt.figure(figsize=(20, 15))
displayed_count = {category: 0 for category in categories}
max_images_per_class = 3
displayed_images = 0

for i in range(len(testX)):
    if testX[i].size == 0:
        continue  # Skip empty images

    true_label = categories[trueY_classes[i]]
    if displayed_count[true_label] < max_images_per_class:
        plt.subplot(5, 6, displayed_images + 1)
        plt.imshow(cv2.resize(testX[i][0], (128, 128)))
        plt.title(f"True: {true_label}\nPred: {categories[predY_classes[i]]}")
        plt.axis('off')

        displayed_count[true_label] += 1
        displayed_images += 1

    if displayed_images >= max_images_per_class * len(categories):
        break

plt.show()


In [ ]:
import os
import cv2
import numpy as np
import time
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import matplotlib.pyplot as plt
from keras.layers import TimeDistributed
from keras.applications import MobileNet
from keras import backend as K
from keras.layers import Layer
import tensorflow as tf

# Attention layer class
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight', shape=(input_shape[-1], 1),
                                 initializer='glorot_uniform', trainable=True)
        self.b = self.add_weight(name='attention_bias', shape=(input_shape[1], 1),
                                 initializer='glorot_uniform', trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

# Function to load and split dataset based on the category (Weaving or Knitting)
def load_dataset_by_category(category_group):
    base_dir = '/content/Fabric FDD/Fabric_classification/Fabric classification'

    # Subcategories for the selected category group (either Weaving or Knitting)
    subcategories = {
        'Weaving_gray': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
        'Weaving_dyed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
        'Weaving_printed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
        'Knitting_gray': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
        'Knitting_dyed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
        'Knitting_printed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective']
    }

    # Filter subcategories based on whether it's "Weaving" or "Knitting"
    selected_subcategories = {k: v for k, v in subcategories.items() if category_group in k}

    images = []
    labels = []
    categories = list(selected_subcategories.keys())
    category_counts = {category: 0 for category in categories}

    for label, category in enumerate(categories):
        for defect in selected_subcategories[category]:
            category_dir = os.path.join(base_dir, category, defect)
            if not os.path.exists(category_dir):
                print(f"Directory {category_dir} does not exist. Skipping this category.")
                continue
            for filename in os.listdir(category_dir):
                if filename.endswith('.jpg'):
                    img = cv2.imread(os.path.join(category_dir, filename))
                    img = cv2.resize(img, (128, 128))  # Resize image to 128x128
                    images.append(img)
                    labels.append(label)
                    category_counts[category] += 1

    images = np.array(images)
    labels = np.array(labels)

    print(f"Image counts per category for {category_group}:", category_counts)

    return images, labels, categories

# Preprocess and train the model for either Weaving or Knitting
def train_and_evaluate(category_group):
    # Load the dataset for the specific category group
    images, labels, categories = load_dataset_by_category(category_group)

    # Ensure that we have images loaded
    if len(images) == 0:
        raise ValueError("No images found. Please check the dataset path and ensure images are available.")

    # Preprocess the images (normalize pixel values)
    images = images.astype('float32') / 255.0

    # Convert labels to one-hot encoding
    labels = to_categorical(labels, num_classes=len(categories))

    # Reshape images for LSTM input
    images = images.reshape(images.shape[0], 1, 128, 128, 3)  # Reshaped for LSTM with 1 timestep

    # Split the dataset into training and test sets
    trainX, testX, trainY, testY = train_test_split(images, labels, test_size=0.2, random_state=42)

    # Load pre-trained MobileNet without the top layers
    feature_extractor = MobileNet(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

    # Freeze the weights of the pre-trained model
    for layer in feature_extractor.layers:
        layer.trainable = False

    # Define the model
    model = Sequential()

    # Use TimeDistributed to apply the MobileNet to each image in the sequence
    model.add(TimeDistributed(feature_extractor, input_shape=(1, 128, 128, 3)))
    model.add(TimeDistributed(GlobalAveragePooling2D()))

    # Add LSTM layer to process the temporal aspect of the data
    model.add(LSTM(128, return_sequences=True))
    model.add(Dropout(0.5))

    # Add the Attention layer
    model.add(AttentionLayer())

    # Add Dense layers for classification
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))

    # Output layer for classification
    model.add(Dense(len(categories), activation='softmax', kernel_regularizer='l2'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # Callbacks for learning rate reduction and early stopping
    lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.5, min_lr=0.00001)
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    start_time = time.time()
    history = model.fit(trainX, trainY, epochs=90, batch_size=32, validation_data=(testX, testY), callbacks=[lr_reduction, early_stopping], verbose=1)
    end_time = time.time()

    # Evaluate the model
    loss, accuracy = model.evaluate(testX, testY, verbose=0)
    print(f'Test Loss for {category_group}:', loss)
    print(f'Test Accuracy for {category_group}:', accuracy)

    # Get predictions for the test set
    predY = model.predict(testX)
    predY_classes = np.argmax(predY, axis=1)
    trueY_classes = np.argmax(testY, axis=1)

    unique_true_labels = np.unique(trueY_classes)

    # Print classification report
    class_report = classification_report(
        trueY_classes, predY_classes,
        target_names=[categories[i] for i in unique_true_labels],
        output_dict=True,
        zero_division=0
    )
    print(f"Classification report for {category_group}:\n", class_report)

    # Print confusion matrix
    conf_matrix = confusion_matrix(trueY_classes, predY_classes)
    print(f"Confusion matrix for {category_group}:\n", conf_matrix)

    accuracy_scores = accuracy_score(trueY_classes, predY_classes)
    f1_scores = f1_score(trueY_classes, predY_classes, average=None)
    print(f"Accuracy Scores for {category_group}: ", accuracy_scores)
    print(f"F1 Scores for {category_group}: ", f1_scores)
    print(f"Average F1 Score for {category_group}: ", np.mean(f1_scores))

    print(f"Total Training Time for {category_group}: {end_time - start_time} seconds")

    # Plot training & validation loss and accuracy
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{category_group} Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{category_group} Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
    plt.show()

# Train and evaluate models for Weaving and Knitting separately
print("Training model for Weaving...")
train_and_evaluate('Weaving')

print("\nTraining model for Knitting...")
train_and_evaluate('Knitting')


In [ ]:
import os
import cv2
import numpy as np
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import matplotlib.pyplot as plt
from keras.utils import to_categorical

# Attention layer class (needed for loading the model)
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight', shape=(input_shape[-1], 1),
                                 initializer='glorot_uniform', trainable=True)
        self.b = self.add_weight(name='attention_bias', shape=(input_shape[1], 1),
                                 initializer='glorot_uniform', trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

# Function to load and split dataset based on the category (Weaving or Knitting)
def load_test_dataset_by_category(category_group):
    base_dir = '/content/Fabric FDD/Fabric_classification/Fabric classification'

    subcategories = {
        'Weaving_gray': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
        'Weaving_dyed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
        'Weaving_printed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
        'Knitting_gray': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
        'Knitting_dyed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
        'Knitting_printed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective']
    }

    selected_subcategories = {k: v for k, v in subcategories.items() if category_group in k}

    images = []
    labels = []
    categories = list(selected_subcategories.keys())
    category_counts = {category: 0 for category in categories}

    for label, category in enumerate(categories):
        for defect in selected_subcategories[category]:
            category_dir = os.path.join(base_dir, category, defect)
            if not os.path.exists(category_dir):
                print(f"Directory {category_dir} does not exist. Skipping this category.")
                continue
            for filename in os.listdir(category_dir):
                if filename.endswith('.jpg'):
                    img = cv2.imread(os.path.join(category_dir, filename))
                    img = cv2.resize(img, (128, 128))  # Resize image to 128x128
                    images.append(img)
                    labels.append(label)
                    category_counts[category] += 1

    images = np.array(images)
    labels = np.array(labels)

    print(f"Image counts per category for {category_group}:", category_counts)

    return images, labels, categories

# Preprocess and evaluate the model for Weaving or Knitting
def test_saved_model(category_group, saved_model_path):
    # Load the dataset for the specific category group
    images, labels, categories = load_test_dataset_by_category(category_group)

    if len(images) == 0:
        raise ValueError("No images found. Please check the dataset path and ensure images are available.")

    # Preprocess the images (normalize pixel values)
    images = images.astype('float32') / 255.0

    # Convert labels to one-hot encoding
    labels = to_categorical(labels, num_classes=len(categories))

    # Reshape images for LSTM input
    images = images.reshape(images.shape[0], 1, 128, 128, 3)  # Reshaped for LSTM with 1 timestep

    # Load the pre-trained model
    model = load_model(saved_model_path, custom_objects={'AttentionLayer': AttentionLayer})

    # Evaluate the model on the test dataset
    loss, accuracy = model.evaluate(images, labels, verbose=0)
    print(f'Test Loss for {category_group}:', loss)
    print(f'Test Accuracy for {category_group}:', accuracy)

    # Get predictions for the test set
    predY = model.predict(images)
    predY_classes = np.argmax(predY, axis=1)
    trueY_classes = np.argmax(labels, axis=1)

    # Print classification report
    unique_true_labels = np.unique(trueY_classes)
    class_report = classification_report(
        trueY_classes, predY_classes,
        target_names=[categories[i] for i in unique_true_labels],
        output_dict=True,
        zero_division=0
    )
    print(f"Classification report for {category_group}:\n", class_report)

    # Print confusion matrix
    conf_matrix = confusion_matrix(trueY_classes, predY_classes)
    print(f"Confusion matrix for {category_group}:\n", conf_matrix)

    # Calculate accuracy and F1 scores
    accuracy_scores = accuracy_score(trueY_classes, predY_classes)
    f1_scores = f1_score(trueY_classes, predY_classes, average=None)
    print(f"Accuracy Scores for {category_group}: ", accuracy_scores)
    print(f"F1 Scores for {category_group}: ", f1_scores)
    print(f"Average F1 Score for {category_group}: ", np.mean(f1_scores))

    # Visualize confusion matrix
    plt.figure(figsize=(8, 6))
    plt.matshow(conf_matrix, cmap='Blues', fignum=1)
    plt.title(f'Confusion Matrix - {category_group}')
    plt.colorbar()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.xticks(np.arange(len(categories)), categories, rotation=90)
    plt.yticks(np.arange(len(categories)), categories)
    plt.show()

# Example usage
print("Testing saved model for Weaving...")
test_saved_model('Weaving', 'path_to_saved_weaving_model.h5')

print("\nTesting saved model for Knitting...")
test_saved_model('Knitting', 'path_to_saved_knitting_model.h5')


In [ ]:
import os
import cv2
import numpy as np
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import matplotlib.pyplot as plt
from keras.utils import to_categorical

# Attention layer class (needed for loading the model)
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight', shape=(input_shape[-1], 1),
                                 initializer='glorot_uniform', trainable=True)
        self.b = self.add_weight(name='attention_bias', shape=(input_shape[1], 1),
                                 initializer='glorot_uniform', trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])


# Function to load and filter dataset based on user selection
def load_test_dataset_by_user_selection(category_group, fabric_class):
    base_dir = '/content/Fabric FDD/Fabric_classification/Fabric classification'

    subcategories = {
        'Weaving': {'gray': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
                    'dyed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
                    'printed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective']},
        'Knitting': {'gray': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
                     'dyed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
                     'printed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective']}
    }

    if category_group not in subcategories or fabric_class not in subcategories[category_group]:
        raise ValueError("Invalid category or sub-category selection.")

    selected_subcategory = subcategories[category_group][fabric_class]

    images = []
    labels = []
    categories = list(selected_subcategory)
    category_counts = {category: 0 for category in categories}

    for label, defect in enumerate(categories):
        category_dir = os.path.join(base_dir, category_group, fabric_class, defect)
        if not os.path.exists(category_dir):
            print(f"Directory {category_dir} does not exist. Skipping this category.")
            continue
        for filename in os.listdir(category_dir):
            if filename.endswith('.jpg'):
                img = cv2.imread(os.path.join(category_dir, filename))
                img = cv2.resize(img, (128, 128))  # Resize image to 128x128
                images.append(img)
                labels.append(label)
                category_counts[defect] += 1

    images = np.array(images)
    labels = np.array(labels)

    print(f"Image counts per defect category for {category_group} - {fabric_class}:", category_counts)

    return images, labels, categories


# Preprocess and evaluate the model based on user input
def test_saved_model_by_user_input(saved_model_path):
    # Ask user to select fabric type (Weaving or Knitting)
    print("What type of fabric do you want to inspect?")
    fabric_type = input("Select: 1) Weaving 2) Knitting: ")

    if fabric_type == '1':
        category_group = 'Weaving'
    elif fabric_type == '2':
        category_group = 'Knitting'
    else:
        print("Invalid selection. Exiting...")
        return

    # Ask user to select fabric class (Gray, Dyed, Printed)
    print(f"What is your fabric class for {category_group}?")
    fabric_class = input("Select: a) gray b) dyed c) printed: ")

    if fabric_class == 'a':
        fabric_class = 'gray'
    elif fabric_class == 'b':
        fabric_class = 'dyed'
    elif fabric_class == 'c':
        fabric_class = 'printed'
    else:
        print("Invalid selection. Exiting...")
        return

    # Load the dataset based on the user's selection
    images, labels, categories = load_test_dataset_by_user_selection(category_group, fabric_class)

    if len(images) == 0:
        raise ValueError("No images found. Please check the dataset path and ensure images are available.")

    # Preprocess the images (normalize pixel values)
    images = images.astype('float32') / 255.0

    # Convert labels to one-hot encoding
    labels = to_categorical(labels, num_classes=len(categories))

    # Reshape images for LSTM input
    images = images.reshape(images.shape[0], 1, 128, 128, 3)

    # Load the pre-trained model
    model = load_model(saved_model_path, custom_objects={'AttentionLayer': AttentionLayer})

    # Evaluate the model on the test dataset
    loss, accuracy = model.evaluate(images, labels, verbose=0)
    print(f'Test Loss for {category_group} - {fabric_class}:', loss)
    print(f'Test Accuracy for {category_group} - {fabric_class}:', accuracy)

    # Get predictions for the test set
    predY = model.predict(images)
    predY_classes = np.argmax(predY, axis=1)
    trueY_classes = np.argmax(labels, axis=1)

    # Print classification report
    unique_true_labels = np.unique(trueY_classes)
    class_report = classification_report(
        trueY_classes, predY_classes,
        target_names=[categories[i] for i in unique_true_labels],
        output_dict=True,
        zero_division=0
    )
    print(f"Classification report for {category_group} - {fabric_class}:\n", class_report)

    # Print confusion matrix
    conf_matrix = confusion_matrix(trueY_classes, predY_classes)
    print(f"Confusion matrix for {category_group} - {fabric_class}:\n", conf_matrix)

    # Calculate accuracy and F1 scores
    accuracy_scores = accuracy_score(trueY_classes, predY_classes)
    f1_scores = f1_score(trueY_classes, predY_classes, average=None)
    print(f"Accuracy Scores for {category_group} - {fabric_class}: ", accuracy_scores)
    print(f"F1 Scores for {category_group} - {fabric_class}: ", f1_scores)
    print(f"Average F1 Score for {category_group} - {fabric_class}: ", np.mean(f1_scores))

    # Visualize confusion matrix
    plt.figure(figsize=(8, 6))
    plt.matshow(conf_matrix, cmap='Blues', fignum=1)
    plt.title(f'Confusion Matrix - {category_group} {fabric_class}')
    plt.colorbar()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.xticks(np.arange(len(categories)), categories, rotation=90)
    plt.yticks(np.arange(len(categories)), categories)
    plt.show()

# Example usage
test_saved_model_by_user_input('path_to_saved_model.h5')


In [ ]:
import os
import cv2
import numpy as np
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.layers import Layer

# Attention layer class (needed for loading the model)
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight', shape=(input_shape[-1], 1),
                                 initializer='glorot_uniform', trainable=True)
        self.b = self.add_weight(name='attention_bias', shape=(input_shape[1], 1),
                                 initializer='glorot_uniform', trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])


# Function to load and filter dataset based on user selection
def load_test_dataset_by_user_selection(category_group, fabric_class):
    base_dir = '/content/Fabric FDD/Fabric_classification/Fabric classification'

    subcategories = {
        'Weaving': {'gray': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
                    'dyed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
                    'printed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective']},
        'Knitting': {'gray': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
                     'dyed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
                     'printed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective']}
    }

    if category_group not in subcategories or fabric_class not in subcategories[category_group]:
        raise ValueError("Invalid category or sub-category selection.")

    selected_subcategory = subcategories[category_group][fabric_class]

    images = []
    labels = []
    categories = list(selected_subcategory)
    category_counts = {category: 0 for category in categories}

    for label, defect in enumerate(categories):
        category_dir = os.path.join(base_dir, category_group, fabric_class, defect)
        if not os.path.exists(category_dir):
            print(f"Directory {category_dir} does not exist. Skipping this category.")
            continue
        for filename in os.listdir(category_dir):
            if filename.endswith('.jpg'):
                img = cv2.imread(os.path.join(category_dir, filename))
                img = cv2.resize(img, (128, 128))  # Resize image to 128x128
                images.append(img)
                labels.append(label)
                category_counts[defect] += 1

    images = np.array(images)
    labels = np.array(labels)

    print(f"Image counts per defect category for {category_group} - {fabric_class}:", category_counts)

    return images, labels, categories


# Preprocess and evaluate the model based on user input
def test_saved_model_by_user_input(saved_model_path):
    # Ask user to select fabric type (Weaving or Knitting)
    print("What type of fabric do you want to inspect?")
    fabric_type = input("Select: 1) Weaving 2) Knitting: ")

    if fabric_type == '1':
        category_group = 'Weaving'
    elif fabric_type == '2':
        category_group = 'Knitting'
    else:
        print("Invalid selection. Exiting...")
        return

    # Ask user to select fabric class (Gray, Dyed, Printed)
    print(f"What is your fabric class for {category_group}?")
    fabric_class = input("Select: a) gray b) dyed c) printed: ")

    if fabric_class == 'a':
        fabric_class = 'gray'
    elif fabric_class == 'b':
        fabric_class = 'dyed'
    elif fabric_class == 'c':
        fabric_class = 'printed'
    else:
        print("Invalid selection. Exiting...")
        return

    # Load the dataset based on the user's selection
    images, labels, categories = load_test_dataset_by_user_selection(category_group, fabric_class)

    if len(images) == 0:
        raise ValueError("No images found. Please check the dataset path and ensure images are available.")

    # Preprocess the images (normalize pixel values)
    images = images.astype('float32') / 255.0

    # Convert labels to one-hot encoding
    labels = to_categorical(labels, num_classes=len(categories))

    # Reshape images for LSTM input
    images = images.reshape(images.shape[0], 1, 128, 128, 3)

    # Load the pre-trained model
    model = load_model(saved_model_path, custom_objects={'AttentionLayer': AttentionLayer})

    # Evaluate the model on the test dataset
    loss, accuracy = model.evaluate(images, labels, verbose=0)
    print(f'Test Loss for {category_group} - {fabric_class}:', loss)
    print(f'Test Accuracy for {category_group} - {fabric_class}:', accuracy)

    # Get predictions for the test set
    predY = model.predict(images)
    predY_classes = np.argmax(predY, axis=1)
    trueY_classes = np.argmax(labels, axis=1)

    # Print classification report
    unique_true_labels = np.unique(trueY_classes)
    class_report = classification_report(
        trueY_classes, predY_classes,
        target_names=[categories[i] for i in unique_true_labels],
        output_dict=True,
        zero_division=0
    )
    print(f"Classification report for {category_group} - {fabric_class}:\n", class_report)

    # Print confusion matrix
    conf_matrix = confusion_matrix(trueY_classes, predY_classes)
    print(f"Confusion matrix for {category_group} - {fabric_class}:\n", conf_matrix)

    # Calculate accuracy and F1 scores
    accuracy_scores = accuracy_score(trueY_classes, predY_classes)
    f1_scores = f1_score(trueY_classes, predY_classes, average=None)
    print(f"Accuracy Scores for {category_group} - {fabric_class}: ", accuracy_scores)
    print(f"F1 Scores for {category_group} - {fabric_class}: ", f1_scores)
    print(f"Average F1 Score for {category_group} - {fabric_class}: ", np.mean(f1_scores))

    # Visualize confusion matrix
    plt.figure(figsize=(8, 6))
    plt.matshow(conf_matrix, cmap='Blues', fignum=1)
    plt.title(f'Confusion Matrix - {category_group} {fabric_class}')
    plt.colorbar()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.xticks(np.arange(len(categories)), categories, rotation=90)
    plt.yticks(np.arange(len(categories)), categories)
    plt.show()

# Example usage
test_saved_model_by_user_input('path_to_saved_model.h5')


In [ ]:
streamlit run fabric_inspection_app.py


In [ ]:
import streamlit as st
import os
import cv2
import numpy as np
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.layers import Layer

# Custom attention layer (needed for loading the model)
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name='attention_weight', shape=(input_shape[-1], 1),
                                 initializer='glorot_uniform', trainable=True)
        self.b = self.add_weight(name='attention_bias', shape=(input_shape[1], 1),
                                 initializer='glorot_uniform', trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        e = tf.keras.backend.tanh(tf.keras.backend.dot(x, self.W) + self.b)
        a = tf.keras.backend.softmax(e, axis=1)
        output = x * a
        return tf.keras.backend.sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])


# Function to load and filter dataset based on user selection
def load_test_dataset_by_user_selection(category_group, fabric_class):
    base_dir = '/content/Fabric FDD/Fabric_classification/Fabric classification'

    subcategories = {
        'Weaving': {'gray': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
                    'dyed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
                    'printed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective']},
        'Knitting': {'gray': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
                     'dyed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective'],
                     'printed': ['stain', 'damage', 'broken thread', 'holes', 'Non defective']}
    }

    if category_group not in subcategories or fabric_class not in subcategories[category_group]:
        raise ValueError("Invalid category or sub-category selection.")

    selected_subcategory = subcategories[category_group][fabric_class]

    images = []
    labels = []
    categories = list(selected_subcategory)
    category_counts = {category: 0 for category in categories}

    for label, defect in enumerate(categories):
        category_dir = os.path.join(base_dir, category_group, fabric_class, defect)
        if not os.path.exists(category_dir):
            st.write(f"Directory {category_dir} does not exist. Skipping this category.")
            continue
        for filename in os.listdir(category_dir):
            if filename.endswith('.jpg'):
                img = cv2.imread(os.path.join(category_dir, filename))
                img = cv2.resize(img, (128, 128))  # Resize image to 128x128
                images.append(img)
                labels.append(label)
                category_counts[defect] += 1

    images = np.array(images)
    labels = np.array(labels)

    st.write(f"Image counts per defect category for {category_group} - {fabric_class}: {category_counts}")

    return images, labels, categories


# Preprocess and evaluate the model based on user input
def test_saved_model_by_user_input(saved_model_path, category_group, fabric_class):
    # Load the dataset based on the user's selection
    images, labels, categories = load_test_dataset_by_user_selection(category_group, fabric_class)

    if len(images) == 0:
        st.error("No images found. Please check the dataset path and ensure images are available.")
        return

    # Preprocess the images (normalize pixel values)
    images = images.astype('float32') / 255.0

    # Convert labels to one-hot encoding
    labels = to_categorical(labels, num_classes=len(categories))

    # Reshape images for LSTM input
    images = images.reshape(images.shape[0], 1, 128, 128, 3)

    # Load the pre-trained model
    model = load_model(saved_model_path, custom_objects={'AttentionLayer': AttentionLayer})

    # Evaluate the model on the test dataset
    loss, accuracy = model.evaluate(images, labels, verbose=0)
    st.write(f'Test Loss for {category_group} - {fabric_class}: {loss}')
    st.write(f'Test Accuracy for {category_group} - {fabric_class}: {accuracy}')

    # Get predictions for the test set
    predY = model.predict(images)
    predY_classes = np.argmax(predY, axis=1)
    trueY_classes = np.argmax(labels, axis=1)

    # Print classification report
    unique_true_labels = np.unique(trueY_classes)
    class_report = classification_report(
        trueY_classes, predY_classes,
        target_names=[categories[i] for i in unique_true_labels],
        output_dict=True,
        zero_division=0
    )
    st.write(f"Classification report for {category_group} - {fabric_class}:")
    st.write(class_report)

    # Print confusion matrix
    conf_matrix = confusion_matrix(trueY_classes, predY_classes)
    st.write(f"Confusion matrix for {category_group} - {fabric_class}:")
    st.write(conf_matrix)

    # Plot confusion matrix
    fig, ax = plt.subplots()
    ax.matshow(conf_matrix, cmap='Blues')
    plt.title(f'Confusion Matrix - {category_group} {fabric_class}')
    plt.colorbar()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.xticks(np.arange(len(categories)), categories, rotation=
